In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
### Load payments
df_payments_TG1 = pd.read_csv(r'./Multirun_NewSearchLogic_Web_run1__TG.csv')
df_payments_CG1 = pd.read_csv(r'./Multirun_NewSearchLogic_Web_run1__CG1.csv')
df_payments_CG2 = pd.read_csv(r'./Multirun_NewSearchLogic_Web_run1__CG2.csv')
df_payments_TG1['group'] = 'TG1'
df_payments_CG1['group'] = 'CG1'
df_payments_CG2['group'] = 'CG2'
df_payments = pd.concat([df_payments_TG1, df_payments_CG1, df_payments_CG2])

# df = df[df['registration date'] < border_install_date]
df_payments['payment date'] = df_payments['payment date'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df_payments['registration date'] = df_payments['registration date'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  
df_payments['payment day'] = df_payments['payment date'].map(lambda x: x.strftime('%Y-%m-%d'))
df_payments['registration day'] = df_payments['registration date'].map(lambda x: x.strftime('%Y-%m-%d'))
df_payments['registration month'] = df_payments['registration date'].map(lambda x: x.strftime('%Y-%m'))
df_payments['time from registration'] = df_payments['payment date'] - df_payments['registration date']
df_payments['days from registration'] = df_payments['time from registration'].map(lambda x: x.days)

In [3]:
df_payments.head()

,ref ID,partner ID,ref date,user ID,registration date,payment ID,payment date,payment amount,revenue in cents,payment service ID,...,place,country,last app,Product Id,group,payment day,registration day,registration month,time from registration,days from registration
0,597.0,12.0,2018-04-09 19:42:49,117795109,2018-04-09 19:42:49,7,2018-04-15 13:17:26,100,179,77,...,NaN,AM,topfacemobappios,NaN,TG1,2018-04-15,2018-04-09,2018-04,5 days 17:34:37,5
1,NaN,NaN,NaN,117780178,2018-04-09 05:48:58,3,2018-04-09 05:59:51,52,26,19,...,sympathy_access,RU,topface,coin.one,TG1,2018-04-09,2018-04-09,2018-04,0 days 00:10:53,0
2,597.0,12.0,2018-04-15 11:05:10,117902087,2018-04-15 11:05:10,2,2018-04-15 12:00:26,326,228,147,...,FromMenu,AE,topfacemobappios,com.topface.topface.premium.week.1,TG1,2018-04-15,2018-04-15,2018-04,0 days 00:55:16,0
3,597.0,12.0,2018-04-05 20:34:55,117698390,2018-04-05 20:34:55,10,2018-04-06 23:39:07,500,485,68,...,super_express_message,UA,standalonetf,premium.pack7,TG1,2018-04-06,2018-04-05,2018-04,1 days 03:04:12,1
4,NaN,NaN,NaN,117644989,2018-04-03 14:29:24,2,2018-04-03 14:30:08,122,62,19,...,sympathy_access,RU,topface,coin.two,TG1,2018-04-03,2018-04-03,2018-04,0 days 00:00:44,0


In [4]:
# 19 - Покупка монеток - PaymentServiceBuyCoins
# 57 - Покупка Premium-статуса - PaymentServicePremium
# 67 - Плата за покупку лайков - PaymentServiceLikes
# 68 - Покупка премиума за бабло - PaymentServiceBuyPremium
# 77 - Плата за покупку бомбы сообщений - Topface.Product.Payment.PaymentServiceBombMessage
# 127 - Автоматическая покупка монет - Topface.Subscription.PaymentWall.AutoRechargeCoinsPaymentService
# 146 - Покупка подписки на монеты - PaymentServiceCoinSubscription
# 147 - Покупка подписки на премиум - PaymentServicePremiumSubscription

In [5]:
# df = df_payments[df_payments['days from registration'] >= 14]

In [6]:
df = df_payments

In [7]:
df.groupby(['group'])['revenue in cents'].sum()

group
CG1     81052
CG2     75801
TG1    114575
Name: revenue in cents, dtype: int64

In [8]:
df['values'] = 1

In [9]:
df.fillna('null').pivot_table(index='place', columns='group', values='revenue in cents', aggfunc=sum).fillna(0).applymap(float)

group,CG1,CG2,TG1
place,,,
BadaboomFilterScreen,439.0,0.0,0.0
Dating,969.0,0.0,276.0
EmptyLikes,2651.0,1875.0,1283.0
FromGuests,534.0,487.0,699.0
FromLikes,1737.0,1341.0,2353.0
FromMenu,1186.0,2762.0,4233.0
FromProfileBuyGifts,0.0,350.0,0.0
FromProfileBuyGiftsunique,0.0,71.0,0.0
FromVipTrialPopup,1834.0,3994.0,1118.0


In [10]:
df.fillna('null').pivot_table(index='place', columns='group', values='values', aggfunc=sum).fillna(0).applymap(float)

group,CG1,CG2,TG1
place,,,
BadaboomFilterScreen,5.0,0.0,0.0
Dating,1.0,0.0,1.0
EmptyLikes,21.0,10.0,5.0
FromGuests,2.0,2.0,1.0
FromLikes,6.0,4.0,5.0
FromMenu,4.0,10.0,11.0
FromProfileBuyGifts,0.0,1.0,0.0
FromProfileBuyGiftsunique,0.0,1.0,0.0
FromVipTrialPopup,3.0,7.0,2.0


In [12]:
df.fillna('null').pivot_table(index='place', columns='group', values='user ID', aggfunc=lambda x: len(set(x))).fillna(0).applymap(float)

group,CG1,CG2,TG1
place,,,
BadaboomFilterScreen,2.0,0.0,0.0
Dating,1.0,0.0,1.0
EmptyLikes,7.0,7.0,4.0
FromGuests,1.0,2.0,1.0
FromLikes,4.0,2.0,4.0
FromMenu,3.0,8.0,9.0
FromProfileBuyGifts,0.0,1.0,0.0
FromProfileBuyGiftsunique,0.0,1.0,0.0
FromVipTrialPopup,3.0,6.0,2.0


In [13]:
df.groupby('group')['user ID'].agg(lambda x: len(set(x)))

group
CG1    200
CG2    210
TG1    224
Name: user ID, dtype: int64

In [11]:
df_first_payments = df.sort_values(['payment date'],ascending=True).groupby('user ID').head(1)
df_first_payments.fillna('null').pivot_table(index='place', columns='group', values='values', aggfunc=sum).fillna(0).applymap(float)
# df_first_payments[df_first_payments['days from registration'] >= 14].fillna('null').pivot_table(index='place', columns='group', values='values', aggfunc=sum).fillna(0).applymap(float)

group,CG1,CG2,TG1
place,,,
BadaboomFilterScreen,1.0,0.0,0.0
Dating,1.0,0.0,0.0
EmptyLikes,2.0,1.0,2.0
FromGuests,0.0,1.0,1.0
FromLikes,2.0,2.0,2.0
FromMenu,3.0,6.0,9.0
FromProfileBuyGiftsunique,0.0,1.0,0.0
FromVipTrialPopup,2.0,5.0,2.0
LeftMenu,4.0,4.0,0.0


In [ ]:
df[df['Product Id'].fillna('none').map(lambda x: bool(re.search('prem',x)))].groupby(['group'])['revenue in cents'].sum()

In [15]:
df[df['Product Id'].fillna('none').map(lambda x: bool(re.search('prem',x)))].groupby(['group'])['user ID'].agg(lambda x: len(set(x)))

group
CG1    64
CG2    57
TG1    74
Name: user ID, dtype: int64